# Step 1: Network Filtering and Processing
---
Run this code block by block to convert a street network in ESRI Shapefile, GeoJSON, or GeoPackage format into a routable and conflated network graph to use in BikewaySim.

Three networks were used in this project. While code to obtain OSM GeoJSONs has been included with downloading OSM notebook, the ABM and HERE networks need to be sourced from the Atlanta Regional Commission and HERE respectively.

#### Import Modules

In [1]:
from pathlib import Path
import geopandas as gpd
import numpy as np

import src.network_filter as network_filter

In [2]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
export_fp = Path(config['project_directory']) / 'Network'
if export_fp.exists() == False:
    export_fp.mkdir()

# General Settings Dictionary
---
The only input needed for this code is the project directory and the settings dictionary. The settings dictionary has 6 inputs that are commented out below. Note that subsequent code blocks were used for other runs, and should be left commented out so it doesn't overwrite the settings dictionary.

In [3]:
settings = {
    'project_filepath': export_fp, # will create a networks folder in this directory
    'project_crs': config['projected_crs_epsg'], # output CRS that all exported files will be in
    'studyarea_filepath': Path(config['studyarea']),
    'studyarea_layer': None, # use if a gpkg or gdb file with multiple layers
    'use_bbox': True # use bounding box of the study area rather than the perimeter
}

#### Import study area:
Specify what area you want to mask the network data by. Only network links that are partially or fully within the study area will be imported. Note: network links are NOT clipped (because this cuts off nodes).

In [4]:
#Adds study area geodataframe to settings dictionary
settings['studyarea'] = network_filter.import_study_area(settings)

The study area is 251.23 square miles.


# Run Network Filter Module to Create General, Road, Bike, and Service Layers
---
From the network_filter.py file run the filter networks function. This will first import the spatial data and then filter the data into a raw, general, road, bike, and serivce layer as long as the filters have been pre-defined within the respective function within 'network_filter.py.'

**Note: If this is the a new network that is not OSM, HERE, or ABM then specify a new filter method by going into the 'network_filter.py' script. Otherwise, none of the links will be filtered into road/bike/service layers.**

**Imported networks will be projected to CRS defined in the settings dictionary**

When entering a new filter, search for the following functions: 'filter_to_general', 'filter_to_road', 'filter_to_bike', and 'filter_to_service.' Then add the network name as a new if statement.

If the filters have been specified (by default there are filters for networks that are named 'osm', 'here', and 'osm') then fill in the dictionary and run the 'filter_networks' function with settings dictionary and network dictionary. The network dictionary has the following keys.

- 'network_name': text name of the network (by defualt accepts 'abm','here','here')
- 'links_fp': the filepath of the original links network data (must have a value)
- 'links_layer': if the file is a geopackage or geodatabase then use this to specify the layer to import
- 'nodes_fp': the filepath of the nodes data if available (replace with None if not available)
- 'nodes_layer': if the file is a geopackage or geodatabase then use this to specify the layer to import
- 'nodes_id': indicates the column name that contains the unique ids for each node (replace with None if not available)
- 'A': indicates the starting node id column name for the links (replace with None if not available)
- 'B': indicates the ending node id column name for the links (replace with None if not available)

# OpenStreetMap (OSM) Filtering
---
OSM is an open-source mapping project (see www.openstreetmap.org). OSM network data can be downloaded using the "Step 0 Downloading OSM.ipynb" Jupyter Notebook.

#### General Layer Filtering
- Remove interstates and interstate ramps (highway = motorway | motorway_link)
- Remove sidewalks unless bicycles explicitly allowed (footway = sidewalk | crossing unless bicycle = yes)

#### Road Layer Filtering
- Keep service roads that have a street name (highway = service and name is not null)
- Keep links with the following keys for the highway tag: 'primary','primary_link','residential','secondary','secondary_link','tertiary','tertiary_link','trunk','trunk_link'

#### Bike Layer Filtering
- Include links with the following keys for the 'highway' tag: 'cycleway','footway','path','pedestrian','steps'

#### Service Layer Filtering
- Include links with the key of 'service' for the 'highway' tag unless they have a name

In [5]:
osm = {
       "network_name": 'osm',
       "edges_filepath": Path(config['project_directory']) / 'OSM_DOWNLOAD/osm_2023.gpkg', 
       "edges_layer": 'edges', 
       "nodes_filepath": Path(config['project_directory']) / 'OSM_DOWNLOAD/osm_2023.gpkg', 
       "nodes_layer": 'nodes',
       "nodes_id": "osmid",
       "A": None,
       "B": None,
       "linkid": 'osmid'
       }

links, nodes = network_filter.filter_networks(settings,osm)

Filtering the osm network.
Provided linkid is not unique.
Generating unique link ids.
There is a nodes layer...
but no reference ids for links.


c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Reference IDs successfully added to links.
osm imported... took 1.32 minutes


In [6]:
links.columns

Index(['osmid', 'timestamp', 'version', 'type', 'highway', 'oneway', 'name',
       'bridge', 'tunnel', 'cycleway', 'service', 'footway', 'sidewalk',
       'bicycle', 'foot', 'access', 'area', 'all_tags', 'geom_type',
       'geometry', 'osm_linkid', 'osm_A', 'osm_B', 'link_type'],
      dtype='object')

In [7]:
#drop anything with highway == na
links = links[~links['highway'].isna()]

## Define link types

#### Remove types
- Remove interstates and interstate ramps
- Remove sidewalks unless bicycles explicitly allowed
- Remove anything with access = "no"
- Remove any unpaved footways or paths surface = "dirt"

In [8]:
#initialize an empty column to populate
links['link_type'] = np.nan

## Conditions

In [9]:
links.columns

Index(['osmid', 'timestamp', 'version', 'type', 'highway', 'oneway', 'name',
       'bridge', 'tunnel', 'cycleway', 'service', 'footway', 'sidewalk',
       'bicycle', 'foot', 'access', 'area', 'all_tags', 'geom_type',
       'geometry', 'osm_linkid', 'osm_A', 'osm_B', 'link_type'],
      dtype='object')

In [10]:
#bicycle riding is allowed (no sometimees implies bike can still be walked)
bike_allowed = links['bicycle'].isin(['yes','permitted','permissive','designated'])
ped_allowed = links['foot'].isin(['yes','permitted','permissive','designated'])

## Restricted access (Interstates, highways, onramps)

In [11]:
#remove restricted access roads
restr_access = links['highway'].isin(['motorway','motorway_link'])
links.loc[restr_access & links['link_type'].isna(),'link_type'] = 'restricted_access_road'

## Private/no access or closed road (explicilty doesn't allow bike/ped)


In [12]:
no_access = (links['access'].isin(['no','private','permit'])) | (links['highway'] == 'disused')
#exception if bicycle/foot has a permissive/yes value
links.loc[no_access & (~bike_allowed | ~ped_allowed) & links['link_type'].isna(),'link_type'] = 'no_access_or_private'

## Bike and Pedestrian Links (no cars allowed)


In [13]:
#sidewalks and crossings (helpful to identify these seperately so they can be removed)
sidewalks = links['footway'].isin(['sidewalk','crossing'])
links.loc[(~bike_allowed & sidewalks) & links['link_type'].isna(),'link_type'] = 'sidewalk_or_crossing'

#ped only links that still provide additional connectivity
#we probably don't want these but they might be essential for connectivity
pedestrian = links['highway'].isin(['path','track','pedestrian','steps','corridor','footway'])
links.loc[(pedestrian & ~bike_allowed) & links['link_type'].isna(),'link_type'] = 'pedestrian'

# dirt trails/paths

#bike specific and ped links where cycling is explicictly tagged as allowed
conditions = links["highway"]=='cycleway'
links.loc[(conditions | bike_allowed) & links['link_type'].isna() ,'link_type'] = 'bike'

## Public and service roads

In [14]:
#parking lot aisles and driveways
parking_and_driveways = links['service'].isin(['parking_aisle','driveway'])
links.loc[parking_and_driveways & links['link_type'].isna(),'link_type'] = 'parking_and_driveways'

#service roads that aren't parking lots or driveways (and don't have names)
conditions = (links['highway'] == 'service') & (links['name'].isnull())
links.loc[conditions & links['link_type'].isna(),'link_type'] = 'service'

#find service links that still have a name
service_links_with_name = (links['highway'] == 'service') & (links['name'].isnull() == False)

#unclassified added 8/14/23 because there are several roads in Atlanta region marked this despite being public roads
osm_filter_method = ['primary','primary_link','residential','secondary','secondary_link',
                    'tertiary','tertiary_link','trunk','trunk_link','unclassified','living_street'
                    'service','living_street'] 
osm_filter_method = links["highway"].isin(osm_filter_method)

#add back in service links with a name
conditions = osm_filter_method | service_links_with_name
links.loc[conditions & links['link_type'].isna(),'link_type'] = 'road'

### NAs
Be sure to check if all links have been classified. Some OSM highway tags aren't clear and might need to be added on a case by case basis (e.g., track, construction, living_street).

In [15]:
links['link_type'].isna().sum()

86

In [16]:
links[links['link_type'].isna()]

,osmid,timestamp,version,type,highway,oneway,name,bridge,tunnel,cycleway,...,foot,access,area,all_tags,geom_type,geometry,osm_linkid,osm_A,osm_B,link_type
3354,437355144,1584323901,4,way,construction,yes,Old Dixie Road,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2228621.429 1322752.351, 2228598.7...",1125585445,6988472181,67485621,NaN
3355,437355144,1584323901,4,way,construction,yes,Old Dixie Road,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2228558.052 1322240.061, 2228561.8...",1125585446,67485621,7297946658,NaN
3381,781664471,1658158245,2,way,construction,yes,Old Dixie Road,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2229215.468 1324231.265, 2229218.0...",1125585472,6988472146,6988472181,NaN
3537,746805178,1603248801,4,way,construction,yes,None,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2229019.813 1324220.187, 2227886.8...",1125585628,6988472147,67376113,NaN
3540,1079499794,1658158245,1,way,construction,yes,Conley Road,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2229247.184 1324289.524, 2227886.7...",1125585631,5444597516,5506569014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163902,601780186,1529948228,1,way,proposed,None,None,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2234305.324 1386553.821, 2234347.4...",1125745993,5718299789,69327393,NaN
189942,66101884,1667470633,5,way,construction,yes,None,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2237400.846 1422530.286, 2237397.4...",1125772033,5174466382,802366812,NaN
189947,533345989,1667470633,2,way,construction,yes,None,None,None,None,...,None,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2237405.420 1422335.202, 2237400.8...",1125772038,5174466383,5174466382,NaN
189949,66101880,1667470633,9,way,construction,yes,None,None,None,None,...,no,None,None,"{""@changeset"": 0, ""@user"": """", ""@way_nodes"": [...",LineString,"LINESTRING (2237409.988 1422232.216, 2237408.2...",1125772040,9252587733,5174466383,NaN


### Export

In [17]:
network_filter.export(links,nodes,'osm',settings,osm)
del links, nodes

Export took 0.76 minutes


# HERE Filtering
---
HERE is licensed data typically used for logistics.

#### General Layer Filtering
- Remove interstates and interstate ramps (controlled access = yes or ramp = yes) that don't allow bikes

#### Road Layer Filtering
- Keep all links that allow automobile access (ar_auto = yes)
- Remove links with a speed limit < 6 mph (speed_cat = 8) as these are driveways

#### Bike Layer Filtering
- All links that do not allow auto access (ar_auto = no)

#### Service Layer Filtering
- All links that allow auto access with a speed limit < 6mph (ar_auto = y and speed_cat = 8)

In [18]:
here = {
       "network_name": 'here', # name for network
       "edges_filepath": Path(config['here_fp']), #filepath for the links
       "edges_layer":None, # layer name for links if gpkg or gdb
       "nodes_filepath": None, # if there is not a nodes file put None
       "nodes_layer": None, # layer name for nodes if gpkg or gdb (put none if no nodes)
       "nodes_id": None, # column name of node id (put none if no node id)
       "A": "REF_IN_ID", # starting node id reference column on links (put none if no reference links)
       "B": "NREF_IN_ID", # starting node id reference column on links (put none if no reference links)
       "linkid": 'LINK_ID'
       }

links, nodes = network_filter.filter_networks(settings,here)

Filtering the here network.
no nodes layer
but links have reference ids


c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


here imported... took 1.17 minutes


In [19]:
#initialize an empty column to populate
links['link_type'] = np.nan

In [20]:
#remove controlled access roads and ramps
controlled_access = (links['CONTRACC'].str.contains('Y')) | (links['RAMP'].str.contains('Y'))
links.loc[controlled_access & links['link_type'].isna(),'link_type'] = 'restricted_access_road'

# road filters
road_links = (links['AR_AUTO'].str.contains('Y')) & (links['SPEED_CAT'].str.contains('8') == False)
links.loc[road_links & links['link_type'].isna(),'link_type'] = 'road'

# bike filters
bike_links = links['AR_AUTO'].str.contains('N')
links.loc[bike_links & links['link_type'].isna(),'link_type'] = 'bike'

# service filters
service_links = (links['AR_AUTO'].str.contains('Y')) & (links['SPEED_CAT'].str.contains('8'))
links.loc[service_links & links['link_type'].isna(),'link_type'] = 'service'

network_filter.export(links,nodes,'here',settings)
del links, nodes

TypeError: export() missing 1 required positional argument: 'network_dict'

# ABM Filtering
---
This is the modeling network used in the Atlanta Regional Commission's Activity Based Model. It was provided by the ARC and is available on request.

#### General Layer Filtering
- Remove all links except for principal arterials, minor arterials, collectors, and local roads (FACTYPE = 10, 11, or 14)

#### Road Layer Filtering
- The filtering done in the previous step was enough, no further filtering needed

#### Bike Layer Filtering
- No bike links present

#### Service Layer Filtering
- No service links present

In [ ]:
abm = {
       "network_name": 'abm',
       "edges_filepath": Path(config['abm_fp']),
       "edges_layer":"DAILY_Link",
       "nodes_filepath": Path(config['abm_fp']),
       "nodes_layer":"DAILY_Node",
       "nodes_id": "N",
       "A": "A",
       "B": "B",
       "linkid": "A_B"
       }

links, nodes = network_filter.filter_networks(settings,abm)

Filtering the abm network.
There is a nodes layer...
and links and nodes have reference ids.
abm imported... took 0.21 minutes


In [ ]:
#need new way to identify these?
#links = remove_directed_links(links,'abm')

#explode and drop level to get rid of multi-index in abm layer
links = links.explode().reset_index(drop=True)

#remove interstates and centroid connectors
abm_road = [10,11,14]
controlled_access_and_centroids = links["FACTYPE"].isin(abm_road)
links.loc[controlled_access_and_centroids,'link_type'] = 'centroids_and_access_links'

network_filter.export(links,nodes,'abm',settings)
del links, nodes

C:\Users\tpassmore6\AppData\Local\Temp\ipykernel_28480\2704670523.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  links = links.explode().reset_index(drop=True)


Export took 0.17 minutes
